# Day 18
## Part 1
This seems so simple that I'm nervous part 2.

In [1]:
def parse_data(s):
    return {tuple(int(x) for x in line.split(',')) 
            for line in s.strip().splitlines()}

test_string = """
2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5
"""

test_data = parse_data(test_string)
test_data

{(1, 2, 2),
 (1, 2, 5),
 (2, 1, 2),
 (2, 1, 5),
 (2, 2, 1),
 (2, 2, 2),
 (2, 2, 3),
 (2, 2, 4),
 (2, 2, 6),
 (2, 3, 2),
 (2, 3, 5),
 (3, 2, 2),
 (3, 2, 5)}

In [2]:
def part_1(data):
    sides = 0
    for x, y, z in data:
        for d in -1, 1:
            sides += sum([
                (x + d, y, z) not in data, 
                (x, y + d, z) not in data,
                (x, y, z + d) not in data
            ])
    return sides

part_1(test_data)

64

In [3]:
data = parse_data(open("input").read())
part_1(data)

4444

## Part 2

Yep. How am I going to do this? I'll have a think during the football.

Right, vectors. Use a location and vector pair to represent each open face. Put them in a graph. Connect them if they're linked and examine the independent subgraphs. Either do some fancy maths I haven't thought of to work out which is the exterior or, more likely, iterate through the sizes of the subgraphs largest first.

In [82]:
import networkx as nx

debug = ((2,2,1), (0, 0, 1))

def part_2(data):
    g = nx.Graph()
    for x, y, z in data:
        for d in -1, 1:
            if (x + d, y, z) not in data:
                g.add_node(((x, y, z), (d, 0, 0)))
            if (x, y + d, z) not in data:
                g.add_node(((x, y, z), (0, d, 0)))
            if (x, y, z + d) not in data:
                g.add_node(((x, y, z), (0, 0, d)))
                
    for p, v in g.nodes:
        x, y, z = p
        vx, vy, vz = v
        if vx != 0:
            for d in -1, 1:
                # Any connected faces on the same cube?
                if (n := (p, (0, d, 0))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("x1")
                if (n := (p, (0, 0, d))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("x2")
                # Any on the same plane?
                if (n := ((x, y + d, z), v)) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("x3")
                if (n := ((x, y, z + d), v)) in g:
                    g.add_edge((p, v), n)
                # Any at a 90 degree angle?
                    if n == debug:
                        print("x4")
                if (n := ((x + d, y - d, z), (0, d, 0))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("x5")
                if (n := ((x + d, y, z - d), (0, 0, d))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("x6")
                        print(p)
                        print(v)
                        print(d)
                        print(n)                    
        if vy != 0:
            for d in -1, 1:
                # Any connected faces on the same cube?
                if (n := (p, (d, 0, 0))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("y1")
                if (n := (p, (0, 0, d))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("y2")
                # Any on the same plane?
                if (n := ((x + d, y, z), v)) in g:
                    if n == debug:
                        print("y3")
                        print(p)
                        print(v)
                        print(d)
                        print(n)
                    g.add_edge((p, v), n)
                if (n := ((x, y, z + d), v)) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("y4")
                        print(p)
                        print(v)
                        print(d)
                        print(n)
                # Any at a 90 degree angle?
                if (n := ((x - d, y + d, z), (d, 0, 0))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("y5")
                if (n := ((x, y + d, z - d), (0, 0, d))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("y6")
                        print(p)
                        print(v)
                        print(d)
                        print(n)
                    
        if vz != 0:
            for d in -1, 1:
                # Any connected faces on the same cube?
                if (n := (p, (d, 0, 0))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("z1")
                if (n := (p, (0, d, 0))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("z1")
                # Any on the same plane?
                if (n := ((x + d, y, z), v)) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("z1")
                if (n := ((x, y + d, z), v)) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("z4")
                # Any at a 90 degree angle?
                if (n := ((x - d, y, z + d), (d, 0, 0))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("z5")
                if (n := ((x, y - d, z + d), (0, d, 0))) in g:
                    g.add_edge((p, v), n)
                    if n == debug:
                        print("z6")
    for subgraph in nx.connected_components(g):
        print(len(subgraph))
    return g
                    

In [72]:
g = part_2(test_data)

x1
z1
x1
z1
31
18
15


I've made a mistake somewhere. Let's create a 3x3 cube with a hole in the middle.

In [83]:
my_test = """
1,1,1
2,1,1
3,1,1
1,2,1
2,2,1
3,2,1
1,3,1
2,3,1
3,3,1
1,1,2
2,1,2
3,1,2
1,2,2
3,2,2
1,3,2
2,3,2
3,3,2
1,1,3
2,1,3
3,1,3
1,2,3
2,2,3
3,2,3
1,3,3
2,3,3
3,3,3
"""
my_data = parse_data(my_test)
g = part_2(my_data)

x6
(1, 2, 2)
(-1, 0, 0)
1
((2, 2, 1), (0, 0, 1))
x6
(1, 2, 2)
(1, 0, 0)
1
((2, 2, 1), (0, 0, 1))
y6
(2, 1, 2)
(0, -1, 0)
1
((2, 2, 1), (0, 0, 1))
y6
(2, 1, 2)
(0, 1, 0)
1
((2, 2, 1), (0, 0, 1))
60


In [33]:
g[((2,2,1), (0, 0, 1))]

AtlasView({((1, 2, 2), (-1, 0, 0)): {}, ((1, 2, 2), (1, 0, 0)): {}, ((2, 1, 2), (0, -1, 0)): {}, ((2, 1, 2), (0, 1, 0)): {}})

In [55]:
((2,2,2), (0,1,0)) in g.nodes

False

In [57]:
((1, 2, 2), (-1, 0, 0)) in g.nodes

True

In [61]:
len(g)

60